In [31]:
from transformers import LongformerModel, LongformerTokenizer

In [32]:
'''
主要预训练模型:
'allenai/longformer-base-4096'
'allenai/longformer-large-4096'
'''
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
print(tokenizer.model_input_names)
print(tokenizer.all_special_tokens)

['input_ids', 'attention_mask']
['<s>', '</s>', '<unk>', '<pad>', '<mask>']


In [33]:
model = LongformerModel.from_pretrained("allenai/longformer-base-4096")
model

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


LongformerModel(
  (embeddings): LongformerEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(4098, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): LongformerEncoder(
    (layer): ModuleList(
      (0): LongformerLayer(
        (attention): LongformerAttention(
          (self): LongformerSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (query_global): Linear(in_features=768, out_features=768, bias=True)
            (key_global): Linear(in_features=768, out_features=768, bias=True)
            (value_global): Linear(in_features=768, out_features=768, bias=True)
          )
          (o

In [34]:
model.config

LongformerConfig {
  "_name_or_path": "allenai/longformer-base-4096",
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "ignore_attention_mask": false,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 4098,
  "model_type": "longformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "sep_token_id": 2,
  "transformers_version": "4.17.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [35]:
for name, para in model.named_parameters():
    print(name)

embeddings.word_embeddings.weight
embeddings.position_embeddings.weight
embeddings.token_type_embeddings.weight
embeddings.LayerNorm.weight
embeddings.LayerNorm.bias
encoder.layer.0.attention.self.query.weight
encoder.layer.0.attention.self.query.bias
encoder.layer.0.attention.self.key.weight
encoder.layer.0.attention.self.key.bias
encoder.layer.0.attention.self.value.weight
encoder.layer.0.attention.self.value.bias
encoder.layer.0.attention.self.query_global.weight
encoder.layer.0.attention.self.query_global.bias
encoder.layer.0.attention.self.key_global.weight
encoder.layer.0.attention.self.key_global.bias
encoder.layer.0.attention.self.value_global.weight
encoder.layer.0.attention.self.value_global.bias
encoder.layer.0.attention.output.dense.weight
encoder.layer.0.attention.output.dense.bias
encoder.layer.0.attention.output.LayerNorm.weight
encoder.layer.0.attention.output.LayerNorm.bias
encoder.layer.0.intermediate.dense.weight
encoder.layer.0.intermediate.dense.bias
encoder.layer.

In [38]:
text = """Transformer-based models are unable to process long sequences due to their self-attention operation, which scales quadratically with the sequence length.
To address this limitation, we introduce the Longformer with an attention mechanism that scales linearly with sequence length, making it easy to process documents of thousands of tokens or longer.
Longformer’s attention mechanism is a drop-in replacement for the standard self-attention and combines a local windowed attention with a task motivated global attention.
Following prior work on long-sequence transformers, we evaluate Longformer on character-level language modeling and achieve state-of-the-art results on text8 and enwik8.
In contrast to most prior work, we also pretrain Longformer and finetune it on a variety of downstream tasks.
Our pretrained Longformer consistently outperforms RoBERTa on long document tasks and sets new state-of-the-art results on Wiki-Hop and TriviaQA."""
text = text * 20
encoded_input = tokenizer(text, return_tensors='pt')
print(encoded_input)
print(encoded_input.keys())
print(encoded_input['input_ids'].shape)  # [1, 4062]

{'input_ids': tensor([[    0, 19163, 22098,  ...,   250,     4,     2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}
dict_keys(['input_ids', 'attention_mask'])
torch.Size([1, 4062])


In [39]:
result = model(**encoded_input)
print(result.last_hidden_state)
print(result.last_hidden_state.shape)  # [1, 4062, 768](限制input_ids最大长度为4098=4096+2)
print(result.pooler_output.shape)

tensor([[[ 0.0174,  0.0906, -0.0091,  ..., -0.1117,  0.0026, -0.0430],
         [-0.1104, -0.1074, -0.1389,  ..., -0.2235, -0.1210, -0.1010],
         [ 0.0685, -0.2601,  0.0280,  ..., -0.8251,  0.0905,  0.3298],
         ...,
         [-0.1008, -0.2864, -0.2561,  ..., -0.2471,  0.2015,  0.3046],
         [ 0.0353,  0.0831, -0.0228,  ..., -0.1881,  0.0023, -0.0578],
         [ 0.1295, -0.1817,  0.0734,  ...,  0.4047, -0.2236,  0.1173]]],
       grad_fn=<SliceBackward0>)
torch.Size([1, 4062, 768])
torch.Size([1, 768])
